In [1]:
import os
import random

In [ ]:
def process_map_to_binary(input_map, outfile):
    free = [".", "G"]
    blocked = ["@", "O", "T", "S", "W"]
    with open(input_map, 'r') as infile, open(outfile, 'w') as out:
        type = infile.readline().strip()
        height_line = infile.readline().strip()
        width_line = infile.readline().strip()
        blank_line = infile.readline().strip()

        height = int(height_line.split()[1])
        width = int(width_line.split()[1])
        # Binary maps in this project use "width height" on the first line
        out.write(f"{width} {height}\n")

        grid = []
        for _ in range(height):
            row_data = infile.readline().strip()
            if len(row_data) != width:
                raise ValueError(f"Map file error: row length {len(row_data)} does not equal width {width}")
            
            row = []
            for c in row_data:
                if c in free:
                    row.append(0)
                else:
                    row.append(1)
            grid.append(row)
            out.write("".join(str(x) for x in row) + "\n")

In [8]:
# test
inpath = r"MAPF_benchmark_maps\arena.map"
outpath = r"Processed_MAPF_maps\arena_binary.map"
process_map_to_binary(inpath, outpath)

In [ ]:
benchmark_dir = r"MAPF_benchmark_maps"
for file in os.listdir(benchmark_dir):
    if file.endswith(".map"):
        inpath = os.path.join(benchmark_dir, file)
        outpath = os.path.join(r"Processed_MAPF_maps", file[:-4] + "_binary.map")
        process_map_to_binary(inpath, outpath)

In [ ]:
def compute_components(grid, width, height):
    """Label connected components of free cells; returns list of lists of (x,y)."""
    comps = []
    comp_id = [[-1 for _ in range(width)] for _ in range(height)]
    dirs = [(1,0),(-1,0),(0,1),(0,-1)]
    for y in range(height):
        for x in range(width):
            if grid[y][x] != 0 or comp_id[y][x] != -1:
                continue
            cid = len(comps)
            stack = [(x, y)]
            cells = []
            while stack:
                cx, cy = stack.pop()
                if comp_id[cy][cx] != -1:
                    continue
                comp_id[cy][cx] = cid
                cells.append((cx, cy))
                for dx, dy in dirs:
                    nx, ny = cx + dx, cy + dy
                    if 0 <= nx < width and 0 <= ny < height and grid[ny][nx] == 0 and comp_id[ny][nx] == -1:
                        stack.append((nx, ny))
            comps.append(cells)
    return comps

In [ ]:
def pick_random_start_goal(map_file, num_agents, outfile):
    with open(map_file, 'r') as mapf, open(outfile, 'w') as out:
        out.write(f"{num_agents}\n")
        width_str, height_str = mapf.readline().strip().split()
        width = int(width_str)
        height = int(height_str)
        print(f"Processing map {map_file} of size {width}x{height} for {num_agents} agents.")
        grid = []
        for _ in range(height):
            row = mapf.readline().strip()
            if len(row) != width:
                raise ValueError(f"Map row length {len(row)} != width {width}")
            grid.append([int(c) for c in row])

        # Precompute connected components of free cells to ensure reachability
        components = compute_components(grid, width, height)
        # Remove any components with fewer than 2 cells (can't host a start/goal pair)
        components = [c for c in components if len(c) >= 2]
        if not components:
            raise ValueError("No connected free regions with at least 2 cells.")

        for agent in range(num_agents):
            # Pick a component that still has at least 2 free cells
            components = [c for c in components if len(c) >= 2]
            if not components:
                raise ValueError("Not enough connected free cells to assign start and goal positions for all agents.")
            comp = random.choice(components)
            start_idx = random.randrange(len(comp))
            start = comp.pop(start_idx)
            goal_idx = random.randrange(len(comp))
            goal = comp.pop(goal_idx)
            out.write(f"{start[0]} {start[1]} {goal[0]} {goal[1]}\n")

In [ ]:
processed_dir = r"Processed_MAPF_maps"
out_dir = r"Random_agent_scenarios"
for file in os.listdir(processed_dir):
    if file.endswith("_binary.map"):
        map_path = os.path.join(processed_dir, file)
        for num_agents in [5, 10, 15, 20, 25, 30, 35, 40]:
            out_path = os.path.join(out_dir, str(num_agents) + "_" + file[:-11] + "_scenario.txt")
            pick_random_start_goal(map_path, num_agents, out_path)

Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 5 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 10 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 15 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 20 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 25 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 30 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 35 agents.
Processing map Processed_MAPF_maps/brc203d_binary.map of size 274x391 for 40 agents.
Processing map Processed_MAPF_maps/lak519d_binary.map of size 168x145 for 5 agents.
Processing map Processed_MAPF_maps/lak519d_binary.map of size 168x145 for 10 agents.
Processing map Processed_MAPF_maps/lak519d_binary.map of size 168x145 for 15 agents.
Processing map Processed_MAPF_maps/lak519d_binary.map of size 168x1

In [48]:
from collections import deque
def load_map(p):
    with open(p) as f:
        w,h = map(int,f.readline().split())
        grid=[list(line.strip()) for line in f if line.strip()]
        print(f"Loaded map {p} of size {w}x{h}")
    return w,h,grid

def load_agents(p):
    data=open(p).read().strip().split()
    n=int(data[0]); vals=list(map(int,data[1:]))
    return [tuple(vals[i:i+4]) for i in range(0,4*n,4)]

def in_bounds(w,h,x,y): return 0<=x<w and 0<=y<h

def free(grid,x,y): 
    if not in_bounds(len(grid[0]),len(grid),x,y): return False
    return grid[y][x]=='0'

def reachable(grid,w,h,sx,sy,gx,gy):
    if not free(grid,sx,sy) or not free(grid,gx,gy): 
        print ("Start or goal is not on a free cell.")
        return False
    if not in_bounds(w,h,sx,sy) or not in_bounds(w,h,gx,gy):
        print ("Start or goal is out of bounds.")
        return False
    q=deque([(sx,sy)]); seen={(sx,sy)}
    while q:
        x,y=q.popleft()
        if (x,y)==(gx,gy): return True
        for dx,dy in ((1,0),(-1,0),(0,1),(0,-1)):
            nx,ny=x+dx,y+dy
            if in_bounds(w,h,nx,ny) and free(grid,nx,ny) and (nx,ny) not in seen:
                seen.add((nx,ny)); q.append((nx,ny))
    return False

In [50]:
dir = r"Random_agent_scenarios"
maps_dir = r"Processed_MAPF_maps"
for file in os.listdir(dir):
    if file.endswith("_scenario.txt"):
        map_file = os.path.join(maps_dir, file.split("_",1)[1][:-13] + "_binary.map")
        scenario_file = os.path.join(dir, file)
        w,h,grid = load_map(map_file)
        agents = load_agents(scenario_file)
        all_reachable = True
        for sx,sy,gx,gy in agents:
            if not reachable(grid,w,h,sx,sy,gx,gy):
                print(f"Agent from ({sx},{sy}) to ({gx},{gy}) is not reachable in map {map_file}")
                all_reachable = False
        if all_reachable:
            print(f"All agents in scenario {scenario_file} are reachable in map {map_file}")

Loaded map Processed_MAPF_maps/den900d_binary.map of size 128x128
All agents in scenario Random_agent_scenarios/35_den900d_scenario.txt are reachable in map Processed_MAPF_maps/den900d_binary.map
Loaded map Processed_MAPF_maps/orz901d_binary.map of size 601x678
All agents in scenario Random_agent_scenarios/25_orz901d_scenario.txt are reachable in map Processed_MAPF_maps/orz901d_binary.map
Loaded map Processed_MAPF_maps/ost101d_binary.map of size 768x704
All agents in scenario Random_agent_scenarios/30_ost101d_scenario.txt are reachable in map Processed_MAPF_maps/ost101d_binary.map
Loaded map Processed_MAPF_maps/den400d_binary.map of size 259x268
All agents in scenario Random_agent_scenarios/10_den400d_scenario.txt are reachable in map Processed_MAPF_maps/den400d_binary.map
Loaded map Processed_MAPF_maps/den600d_binary.map of size 354x169
All agents in scenario Random_agent_scenarios/40_den600d_scenario.txt are reachable in map Processed_MAPF_maps/den600d_binary.map
Loaded map Processed

In [43]:
w,h,grid=load_map('Processed_MAPF_maps/den203d_binary.map')
for scen in ['Random_agent_scenarios/5_den203d_scenario.txt',
             'Random_agent_scenarios/10_den203d_scenario.txt',
             'Random_agent_scenarios/15_den203d_scenario.txt',
             'Random_agent_scenarios/20_den203d_scenario.txt',
             'Random_agent_scenarios/25_den203d_scenario.txt',
             'Random_agent_scenarios/30_den203d_scenario.txt',
             'Random_agent_scenarios/35_den203d_scenario.txt',
             'Random_agent_scenarios/40_den203d_scenario.txt']:
    bad=[]
    for i,(sx,sy,gx,gy) in enumerate(load_agents(scen)):
        if not reachable(grid,w,h,sx,sy,gx,gy):
            bad.append((i,sx,sy,gx,gy))
    print(scen, 'unreachable', len(bad), 'examples', bad[:3])

Loaded map Processed_MAPF_maps/den203d_binary.map of size 77x93
Random_agent_scenarios/5_den203d_scenario.txt unreachable 1 examples [(2, 83, 27, 20, 43)]
Random_agent_scenarios/10_den203d_scenario.txt unreachable 3 examples [(1, 85, 12, 14, 43), (4, 43, 57, 79, 22), (5, 74, 26, 87, 28)]
Random_agent_scenarios/15_den203d_scenario.txt unreachable 3 examples [(1, 13, 39, 82, 27), (12, 15, 31, 85, 31), (14, 29, 4, 84, 19)]
Start or goal is not on a free cell.
Start or goal is not on a free cell.
Start or goal is not on a free cell.
Start or goal is not on a free cell.
Start or goal is not on a free cell.
Start or goal is not on a free cell.
Random_agent_scenarios/20_den203d_scenario.txt unreachable 7 examples [(0, 53, 58, 83, 28), (3, 64, 25, 18, 46), (6, 3, 25, 29, 47)]
Random_agent_scenarios/25_den203d_scenario.txt unreachable 5 examples [(2, 10, 29, 80, 15), (5, 51, 42, 84, 12), (6, 79, 13, 42, 63)]
Random_agent_scenarios/30_den203d_scenario.txt unreachable 6 examples [(2, 78, 25, 44, 

In [44]:
w,h,grid=load_map('Processed_MAPF_maps/arena_binary.map')
for scen in ['Random_agent_scenarios/5_arena_scenario.txt',
             'Random_agent_scenarios/10_arena_scenario.txt',
             'Random_agent_scenarios/15_arena_scenario.txt',
             'Random_agent_scenarios/20_arena_scenario.txt',
             'Random_agent_scenarios/25_arena_scenario.txt',
             'Random_agent_scenarios/30_arena_scenario.txt',
             'Random_agent_scenarios/35_arena_scenario.txt',
             'Random_agent_scenarios/40_arena_scenario.txt']:
    bad=[]
    for i,(sx,sy,gx,gy) in enumerate(load_agents(scen)):
        if not reachable(grid,w,h,sx,sy,gx,gy):
            bad.append((i,sx,sy,gx,gy))
    print(scen, 'unreachable', len(bad), 'examples', bad[:3])

Loaded map Processed_MAPF_maps/arena_binary.map of size 49x49
Random_agent_scenarios/5_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/10_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/15_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/20_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/25_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/30_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/35_arena_scenario.txt unreachable 0 examples []
Random_agent_scenarios/40_arena_scenario.txt unreachable 0 examples []
